In [1]:
import pandas as pd
import numpy as np

In [2]:
diabetes = pd.read_csv('data/diabetes_data.csv')
diabetes.head()
diab_df = diabetes.copy()

dupl_columns = list(diab_df.columns)
#dupl_columns.remove('id')
mask = diab_df.duplicated(subset=dupl_columns)
diab_duplicates = diab_df[mask]
print(f'Число найденных дубликатов: {diab_duplicates.shape[0]}')

diab_dedupped = diab_df.drop_duplicates(dupl_columns)
print(f'Результирующее число записей: {diab_dedupped.shape[0]}')

Число найденных дубликатов: 10
Результирующее число записей: 768


In [3]:
#список неинформативных признаков
low_information_cols = [] 

#цикл по всем столбцам
for col in diab_dedupped.columns:
    #наибольшая относительная частота в признаке
    top_freq = diab_dedupped[col].value_counts(normalize=True).max()
    #доля уникальных значений от размера признака
    nunique_ratio = diab_dedupped[col].nunique() / diab_dedupped[col].count()
    # сравниваем наибольшую частоту с порогом
    if top_freq > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    # сравниваем долю уникальных значений с порогом
    if nunique_ratio > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')

Gender: 100.0% одинаковых значений


In [ ]:
information_diab_dedupped = diab_dedupped.drop(low_information_cols, axis=1)
print(f'Результирующее число признаков: {information_diab_dedupped.shape[1]}')
print(information_diab_dedupped)

In [ ]:
def IsNullValue(val):
    if val==0:
        return np.nan
    else:
        return val
    
    
for col in information_diab_dedupped.columns:
    information_diab_dedupped[col] = information_diab_dedupped[col].apply(IsNullValue)
    
#print(information_diab_dedupped['Insulin'].isnull().value_counts(normalize=True))    
print(information_diab_dedupped)


In [6]:
cols_null_percent = information_diab_dedupped.isnull().mean() * 100
cols_with_null = cols_null_percent[cols_null_percent>0].sort_values(ascending=False)
print(cols_with_null)
#information_diab_dedupped.drop('Outcome',axis=1)


#print(thresh, information_diab_dedupped.shape[0])

drop_data = information_diab_dedupped.copy()
thresh = drop_data.shape[0]*0.7
drop_data = drop_data.dropna(thresh=thresh,axis=1)
print(drop_data.shape[1])

print(drop_data.isnull().mean())

Outcome          65.104167
Insulin          48.697917
SkinThickness    29.557292
Pregnancies      14.453125
BloodPressure     4.557292
BMI               1.432292
Glucose           0.651042
dtype: float64
7
Pregnancies                 0.144531
Glucose                     0.006510
BloodPressure               0.045573
SkinThickness               0.295573
BMI                         0.014323
DiabetesPedigreeFunction    0.000000
Age                         0.000000
dtype: float64


In [44]:
m = drop_data.shape[1]
drop_data = drop_data.dropna(thresh=m-2,axis=0)
print(drop_data.shape[0])

752
